# Convolutional model implementtion 
## LeNet 5 
### Using Con2d as a convolutional layer and Linear as a linear layer from torch

In [56]:
import torch
import torch.nn as nn #parent
import torch.nn.functional as F #activation func, max pooling funcs
from torch.utils.data import ConcatDataset
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

## Creating model

In [57]:
class LeNet_5(nn.Module):
    def __init__(self):
        super(LeNet_5, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # x = F.max_pool2d(F.relu(self.conv2(x)), 2)

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features = num_features * s
        return num_features
    


## Downloading datasets from torchvision and applying transforms to it

In [58]:
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)

x = torch.stack([sample[0] for sample in ConcatDataset([trainset])])
mean = torch.mean(x, dim=(0,2,3))
std = torch.std(x, dim=(0,2,3)) #standart deviation 

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, std)])

trainset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)

Files already downloaded and verified
Files already downloaded and verified


## Organizing data into batches

In [59]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

## Adding optimizer

In [60]:
net = LeNet_5()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training model

In [61]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.130
[1,  4000] loss: 1.791
[1,  6000] loss: 1.640
[1,  8000] loss: 1.558
[1, 10000] loss: 1.508
[1, 12000] loss: 1.453
[2,  2000] loss: 1.400
[2,  4000] loss: 1.382
[2,  6000] loss: 1.346
[2,  8000] loss: 1.317
[2, 10000] loss: 1.278
[2, 12000] loss: 1.265
Finished Training
